In [ ]:
# STL Processing - Emulating Blender Actions

Convert pre-processed GeoTIFF heightmap to 3D mesh for STL export

In [ ]:
import numpy as np
import rasterio
from scipy.interpolate import RectBivariateSpline
import struct
import json

def import_raster_as_heightmap(input_tiff, scale_z=1.0):
    """
    Import a raster file as a heightmap (elevation data).
    
    Parameters:
    -----------
    input_tiff : str
        Path to the input pre-processed TIFF file
    scale_z : float
        Vertical exaggeration factor for elevation (default: 1.0)
    
    Returns:
    --------
    dict : Dictionary with heightmap data and metadata
        - 'heightmap': numpy array of elevation values
        - 'width': raster width in pixels
        - 'height': raster height in pixels
        - 'bounds': geographic bounds (minx, miny, maxx, maxy)
        - 'resolution': (pixel_size_x, pixel_size_y)
        - 'crs': coordinate reference system
        - 'min_elevation': minimum elevation value
        - 'max_elevation': maximum elevation value
    """
    
    with rasterio.open(input_tiff) as src:
        # Read elevation data (first band)
        heightmap = src.read(1)
        
        # Apply vertical exaggeration
        heightmap = heightmap * scale_z
        
        # Get metadata
        metadata = {
            'heightmap': heightmap,
            'width': src.width,
            'height': src.height,
            'bounds': src.bounds,
            'resolution': src.res,
            'crs': str(src.crs),
            'min_elevation': float(np.nanmin(heightmap)),
            'max_elevation': float(np.nanmax(heightmap)),
            'transform': src.transform
        }
    
    return metadata


def heightmap_to_mesh(heightmap_data, resolution=1.0, smooth=False):
    """
    Convert a heightmap to a 3D mesh in Blender-style format.
    Emulates Blender's "Landscape" add-on behavior.
    
    Parameters:
    -----------
    heightmap_data : dict
        Dictionary returned from import_raster_as_heightmap()
    resolution : float
        Mesh resolution (1.0 = use all pixels, 2.0 = skip every other pixel, etc.)
    smooth : bool
        Apply smoothing to the mesh (default: False)
    
    Returns:
    --------
    dict : Dictionary with mesh data
        - 'vertices': list of [x, y, z] vertex coordinates
        - 'faces': list of [v1, v2, v3] face triangles
        - 'vertex_count': total number of vertices
        - 'face_count': total number of faces
        - 'bounds': mesh bounding box
    """
    
    heightmap = heightmap_data['heightmap']
    pixel_width, pixel_height = heightmap_data['resolution']
    
    # Apply resolution decimation if needed
    step = int(resolution)
    if step > 1:
        heightmap = heightmap[::step, ::step]
    
    height, width = heightmap.shape
    
    # Generate vertex grid
    vertices = []
    
    # Create mesh vertices from heightmap
    for y in range(height):
        for x in range(width):
            # Geographic coordinates
            world_x = x * pixel_width * resolution
            world_y = y * pixel_height * resolution
            world_z = heightmap[y, x]
            
            vertices.append([world_x, world_y, world_z])
    
    # Generate faces (triangles from quad grid)
    faces = []
    
    for y in range(height - 1):
        for x in range(width - 1):
            # Current quad indices
            v0 = y * width + x
            v1 = y * width + (x + 1)
            v2 = (y + 1) * width + x
            v3 = (y + 1) * width + (x + 1)
            
            # Split quad into two triangles
            faces.append([v0, v1, v2])
            faces.append([v1, v3, v2])
    
    # Calculate bounding box
    vertices_array = np.array(vertices)
    bounds = {
        'min_x': float(np.min(vertices_array[:, 0])),
        'max_x': float(np.max(vertices_array[:, 0])),
        'min_y': float(np.min(vertices_array[:, 1])),
        'max_y': float(np.max(vertices_array[:, 1])),
        'min_z': float(np.min(vertices_array[:, 2])),
        'max_z': float(np.max(vertices_array[:, 2]))
    }
    
    mesh_data = {
        'vertices': vertices,
        'faces': faces,
        'vertex_count': len(vertices),
        'face_count': len(faces),
        'bounds': bounds
    }
    
    return mesh_data


# Example usage:
# heightmap = import_raster_as_heightmap('output_cleaned.tif', scale_z=1.0)
# print(f"Heightmap loaded: {heightmap['width']}x{heightmap['height']}")
# 
# mesh = heightmap_to_mesh(heightmap, resolution=1.0, smooth=False)
# print(f"Mesh created: {mesh['vertex_count']} vertices, {mesh['face_count']} faces")

## Convert Heightmap to 3D Mesh

Creates triangulated mesh from heightmap elevation data:
- **Vertices**: Grid of 3D points (x, y, z) from heightmap
- **Faces**: Triangulated surface connecting vertices
- **Mesh Resolution**: Option to decimate for performance
- **Output**: Ready for STL export or Blender import

In [ ]:
import struct

def export_mesh_to_stl(mesh_data, output_stl, ascii_format=False):
    """
    Export 3D mesh to STL format for 3D printing or CAD applications.
    
    Parameters:
    -----------
    mesh_data : dict
        Dictionary returned from heightmap_to_mesh()
    output_stl : str
        Path to output STL file
    ascii_format : bool
        If True, export as ASCII STL (larger file)
        If False, export as binary STL (smaller, faster) - default
    
    Returns:
    --------
    dict : Export info (file path, file size, vertex/face count)
    """
    
    vertices = mesh_data['vertices']
    faces = mesh_data['faces']
    
    if ascii_format:
        # ASCII STL format
        with open(output_stl, 'w') as f:
            f.write("solid geometry\n")
            
            for face in faces:
                v0, v1, v2 = face
                p0 = np.array(vertices[v0])
                p1 = np.array(vertices[v1])
                p2 = np.array(vertices[v2])
                
                # Calculate normal vector
                edge1 = p1 - p0
                edge2 = p2 - p0
                normal = np.cross(edge1, edge2)
                norm = np.linalg.norm(normal)
                if norm > 0:
                    normal = normal / norm
                
                f.write(f"  facet normal {normal[0]:.6e} {normal[1]:.6e} {normal[2]:.6e}\n")
                f.write("    outer loop\n")
                f.write(f"      vertex {p0[0]:.6e} {p0[1]:.6e} {p0[2]:.6e}\n")
                f.write(f"      vertex {p1[0]:.6e} {p1[1]:.6e} {p1[2]:.6e}\n")
                f.write(f"      vertex {p2[0]:.6e} {p2[1]:.6e} {p2[2]:.6e}\n")
                f.write("    endloop\n")
                f.write("  endfacet\n")
            
            f.write("endsolid geometry\n")
    else:
        # Binary STL format (more efficient)
        with open(output_stl, 'wb') as f:
            # 80-byte header
            header = b'Binary STL file - Generated from heightmap' + b'\0' * (80 - 42)
            f.write(header)
            
            # Number of triangles
            f.write(struct.pack('<I', len(faces)))
            
            for face in faces:
                v0, v1, v2 = face
                p0 = np.array(vertices[v0], dtype=np.float32)
                p1 = np.array(vertices[v1], dtype=np.float32)
                p2 = np.array(vertices[v2], dtype=np.float32)
                
                # Calculate normal
                edge1 = p1 - p0
                edge2 = p2 - p0
                normal = np.cross(edge1, edge2)
                norm = np.linalg.norm(normal)
                if norm > 0:
                    normal = normal / norm
                else:
                    normal = np.array([0, 0, 1], dtype=np.float32)
                
                # Write normal
                f.write(struct.pack('<fff', *normal.astype(np.float32)))
                
                # Write vertices
                f.write(struct.pack('<fff', *p0))
                f.write(struct.pack('<fff', *p1))
                f.write(struct.pack('<fff', *p2))
                
                # Attribute byte count
                f.write(struct.pack('<H', 0))
    
    # Get file size
    file_size = os.path.getsize(output_stl)
    
    return {
        'output_path': output_stl,
        'file_size_mb': round(file_size / (1024 * 1024), 3),
        'format': 'ASCII' if ascii_format else 'Binary',
        'vertex_count': mesh_data['vertex_count'],
        'face_count': mesh_data['face_count']
    }

# Example workflow:
# 1. Import raster as heightmap
# heightmap = import_raster_as_heightmap('output_cleaned.tif', scale_z=1.0)
# print(f"Loaded: {heightmap['width']}x{heightmap['height']} - Elevation range: {heightmap['min_elevation']:.1f}m to {heightmap['max_elevation']:.1f}m")
#
# 2. Convert to mesh
# mesh = heightmap_to_mesh(heightmap, resolution=1.0)
# print(f"Mesh created: {mesh['vertex_count']} vertices, {mesh['face_count']} faces")
#
# 3. Export to STL
# stl_info = export_mesh_to_stl(mesh, 'output_model.stl', ascii_format=False)
# print(f"STL exported: {stl_info['output_path']}")

import os

## Better Approach: Displacement-Based Solid Model for 3D Printing

Instead of direct mesh conversion, use displacement mapping with a subdivided grid for better control and solid geometry creation.

In [ ]:
from scipy.interpolate import RectBivariateSpline

def create_displaced_grid_model(input_tiff, grid_subdivisions=1000, displacement_scale=0.0002, 
                                base_thickness=5.0, model_size=100.0):
    """
    Create a 3D printable solid model using displacement mapping.
    This approach provides better control over geometry and creates a solid model suitable for FDM printing.
    
    Parameters:
    -----------
    input_tiff : str
        Path to the heightmap TIFF file
    grid_subdivisions : int
        Number of subdivisions in the grid (default: 1000x1000)
        Higher values = more detail but larger file
    displacement_scale : float
        Scale factor for displacement height (default: 0.0002)
        Adjust to make peaks and troughs visible
    base_thickness : float
        Thickness of the solid base in model units (default: 5.0)
    model_size : float
        Size of the model in XY plane (default: 100.0 units)
    
    Returns:
    --------
    dict : Dictionary with solid mesh data ready for 3D printing
        - 'vertices': vertex coordinates
        - 'faces': triangle faces
        - 'top_vertices': count of top surface vertices
        - 'has_base': boolean indicating solid model
        - 'displacement_range': min/max displacement values
    """
    
    # Load heightmap
    with rasterio.open(input_tiff) as src:
        heightmap = src.read(1).astype(float)
        
        # Normalize heightmap to 0-1 range
        h_min = np.nanmin(heightmap)
        h_max = np.nanmax(heightmap)
        heightmap_normalized = (heightmap - h_min) / (h_max - h_min)
        
        # Create interpolation function for smooth sampling
        h, w = heightmap.shape
        y_coords = np.arange(h)
        x_coords = np.arange(w)
        interpolator = RectBivariateSpline(y_coords, x_coords, heightmap_normalized, kx=3, ky=3)
    
    # Create subdivided grid
    grid_size = grid_subdivisions + 1
    u = np.linspace(0, 1, grid_size)
    v = np.linspace(0, 1, grid_size)
    
    vertices = []
    
    # Generate top surface vertices with displacement
    print(f"Generating {grid_size}x{grid_size} displaced grid...")
    
    for i, v_coord in enumerate(v):
        for j, u_coord in enumerate(u):
            # Map UV to model space
            x = u_coord * model_size
            y = v_coord * model_size
            
            # Sample heightmap at UV coordinates
            sample_y = v_coord * (h - 1)
            sample_x = u_coord * (w - 1)
            displacement_value = interpolator(sample_y, sample_x)[0, 0]
            
            # Apply displacement scaling
            z = displacement_value * displacement_scale * model_size + base_thickness
            
            vertices.append([x, y, z])
    
    # Generate bottom surface vertices (flat base)
    for i, v_coord in enumerate(v):
        for j, u_coord in enumerate(u):
            x = u_coord * model_size
            y = v_coord * model_size
            z = 0.0  # Bottom at z=0
            
            vertices.append([x, y, z])
    
    # Generate faces for top surface
    faces = []
    
    print("Generating top surface faces...")
    for i in range(grid_subdivisions):
        for j in range(grid_subdivisions):
            # Top surface indices
            v0 = i * grid_size + j
            v1 = i * grid_size + (j + 1)
            v2 = (i + 1) * grid_size + j
            v3 = (i + 1) * grid_size + (j + 1)
            
            # Two triangles per quad
            faces.append([v0, v1, v2])
            faces.append([v1, v3, v2])
    
    # Generate faces for bottom surface (inverted normals)
    print("Generating bottom surface faces...")
    bottom_offset = grid_size * grid_size
    
    for i in range(grid_subdivisions):
        for j in range(grid_subdivisions):
            v0 = bottom_offset + i * grid_size + j
            v1 = bottom_offset + i * grid_size + (j + 1)
            v2 = bottom_offset + (i + 1) * grid_size + j
            v3 = bottom_offset + (i + 1) * grid_size + (j + 1)
            
            # Inverted winding order for bottom
            faces.append([v0, v2, v1])
            faces.append([v1, v2, v3])
    
    # Generate side walls to close the mesh (making it watertight/manifold)
    print("Generating side walls...")
    
    # Front wall (y=0)
    for j in range(grid_subdivisions):
        top_v0 = j
        top_v1 = j + 1
        bottom_v0 = bottom_offset + j
        bottom_v1 = bottom_offset + j + 1
        
        faces.append([top_v0, bottom_v0, top_v1])
        faces.append([top_v1, bottom_v0, bottom_v1])
    
    # Back wall (y=max)
    back_row = grid_subdivisions * grid_size
    for j in range(grid_subdivisions):
        top_v0 = back_row + j
        top_v1 = back_row + j + 1
        bottom_v0 = bottom_offset + back_row + j
        bottom_v1 = bottom_offset + back_row + j + 1
        
        faces.append([top_v0, top_v1, bottom_v0])
        faces.append([top_v1, bottom_v1, bottom_v0])
    
    # Left wall (x=0)
    for i in range(grid_subdivisions):
        top_v0 = i * grid_size
        top_v1 = (i + 1) * grid_size
        bottom_v0 = bottom_offset + i * grid_size
        bottom_v1 = bottom_offset + (i + 1) * grid_size
        
        faces.append([top_v0, top_v1, bottom_v0])
        faces.append([top_v1, bottom_v1, bottom_v0])
    
    # Right wall (x=max)
    right_col = grid_subdivisions
    for i in range(grid_subdivisions):
        top_v0 = i * grid_size + right_col
        top_v1 = (i + 1) * grid_size + right_col
        bottom_v0 = bottom_offset + i * grid_size + right_col
        bottom_v1 = bottom_offset + (i + 1) * grid_size + right_col
        
        faces.append([top_v0, bottom_v0, top_v1])
        faces.append([top_v1, bottom_v0, bottom_v1])
    
    # Calculate displacement statistics
    vertices_array = np.array(vertices)
    displacement_range = {
        'min_z': float(np.min(vertices_array[:grid_size*grid_size, 2])),
        'max_z': float(np.max(vertices_array[:grid_size*grid_size, 2])),
        'relief_height': float(np.max(vertices_array[:grid_size*grid_size, 2]) - base_thickness)
    }
    
    print(f"Mesh complete: {len(vertices)} vertices, {len(faces)} faces")
    print(f"Displacement range: {displacement_range['min_z']:.2f} to {displacement_range['max_z']:.2f}")
    
    return {
        'vertices': vertices,
        'faces': faces,
        'vertex_count': len(vertices),
        'face_count': len(faces),
        'top_vertices': grid_size * grid_size,
        'has_base': True,
        'is_manifold': True,
        'displacement_range': displacement_range,
        'parameters': {
            'grid_subdivisions': grid_subdivisions,
            'displacement_scale': displacement_scale,
            'base_thickness': base_thickness,
            'model_size': model_size
        }
    }


# Example usage - Complete pipeline for 3D printing:
# 
# # Create solid displaced model
# solid_mesh = create_displaced_grid_model(
#     'output_cleaned.tif',
#     grid_subdivisions=1000,
#     displacement_scale=0.0002,
#     base_thickness=5.0,
#     model_size=100.0
# )
# 
# # Export to STL
# stl_result = export_mesh_to_stl(solid_mesh, 'terrain_model_3dprint.stl', ascii_format=False)
# print(f"3D printable model exported: {stl_result['file_size_mb']} MB")

## Why This Approach is Better for 3D Printing

**Advantages:**
1. **Solid/Manifold Geometry**: Creates watertight mesh with base and walls (required for slicers)
2. **Fine Control**: Adjust `grid_subdivisions` for detail vs. file size balance
3. **UV-Based Displacement**: Smooth interpolation from heightmap texture
4. **Visible Relief**: `displacement_scale` parameter makes terrain features prominent
5. **Printable Scale**: `model_size` and `base_thickness` ensure practical dimensions

**Parameter Tuning:**
- `grid_subdivisions=1000`: Good balance (1M vertices). Use 500 for faster testing, 2000+ for maximum detail
- `displacement_scale=0.0002`: Adjust based on terrain variation. Higher = more exaggerated relief
- `base_thickness=5.0`: Minimum thickness for stability during printing
- `model_size=100.0`: Physical size in mm (100mm = 10cm square model)

**vs. Direct Conversion:**
- Direct method creates surface-only mesh (hollow)
- Displacement approach creates solid printable object
- Better for FDM/FFF printing with bases and supports

In [ ]:
# Complete workflow example - from preprocessed TIFF to printable STL

# Option 1: Quick surface mesh (hollow - not ideal for printing)
# heightmap = import_raster_as_heightmap('output_cleaned.tif', scale_z=1.0)
# mesh_surface = heightmap_to_mesh(heightmap, resolution=2.0)
# export_mesh_to_stl(mesh_surface, 'terrain_surface.stl')

# Option 2: RECOMMENDED - Solid displaced model (watertight, printable)
solid_model = create_displaced_grid_model(
    input_tiff='output_cleaned.tif',
    grid_subdivisions=1000,      # 1000x1000 grid as you suggested
    displacement_scale=0.0002,   # Your suggested scale factor
    base_thickness=5.0,          # 5mm solid base
    model_size=100.0             # 100mm x 100mm footprint
)

# Export as binary STL (smaller file size)
result = export_mesh_to_stl(solid_model, 'terrain_3d_printable.stl', ascii_format=False)

print(f"✓ 3D Printable Model Created!")
print(f"  File: {result['output_path']}")
print(f"  Size: {result['file_size_mb']} MB")
print(f"  Geometry: {result['vertex_count']:,} vertices, {result['face_count']:,} faces")
print(f"  Relief height: {solid_model['displacement_range']['relief_height']:.2f} mm")
print(f"  Manifold: {solid_model['is_manifold']}")
print(f"\nReady for slicing in Cura, PrusaSlicer, etc.")